# Metadata

**L1 Taxonomy** - Computing Paradigms

**L2 Taxonomy** - Imperative Programming

**Subtopic** - Control Structures and Flow Control

**Use Case** - Develop a Python module to construct and visualize control flow graphs from Python source code using abstract syntax trees.

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
```


# Prompt

You are building a software security team, which needs to analyze Python code for potential vulnerabilities by tracking execution paths. You are supposed to build a tool that constructs control flow graphs from Python source code and identifies suspicious patterns like unreachable code segments, infinite loops, and exception bypass paths.

Your task is to create a Python module that parses source code using Abstract Syntax Trees (AST) and generates a simplified control flow representation. The tool should handle complex nested structures and provide metrics about code complexity.

Your solution should be as a single file with separate driver function and must be testable.

## Input Format

The input consists of:
- source_lines: List of strings representing Python source code (each line contains at most 100 characters)

## Output Format

Output should contain a tuple with:
- blocks_count: Total number of basic blocks identified
- decision_points: Number of decision points (if/elif/else, while, for, try/except)
- max_depth: Maximum nesting depth
- unreachable_blocks: Number of unreachable block IDs (if any, otherwise "NONE")
- loop_status: "INFINITE" if potential infinite loop detected, otherwise "FINITE"

## Examples

### Example 1:

Input:

```
def calculate(x):
    if x > 0:
        return x * 2
    else:
        return 0
    print("unreachable")
```

Output:

```
(5, 2, 2, 4, FINITE)
```

### Example 2:

Input:

```
def process_data(items):
    for item in items:
        if not item:
            continue
        try:
            result = item.process()
        except:
            pass
    return "done"
```

Output:

```
(8, 4, 3, NONE, FINITE)
```

### Example 3:

Input:

```
def loop_test():
    x = 1
    while x > 0:
        x += 1
    return x
```

Output:

```
(5, 1, 2, NONE, INFINITE)
```

## Function Signature

```python
def analyze_control_flow(source_lines):
    """
    Analyze Python source code and return control flow metrics.
    
    Args:
        source_lines: List of strings representing Python source code
        
    Returns:
        tuple: (blocks_count, decision_points, max_depth, unreachable_blocks, loop_status)
    """
    pass
```


# Requirements

## Explicit Requirements
1. Parse Python source code using the ast module
2. Identify basic blocks (sequential statements with single entry/exit). Block IDs are assigned sequentially starting from 0.
3. Count decision points (if/elif/else, loops, try/except blocks). Decision points include all branching constructs.
4. Calculate maximum nesting depth
5. Detect unreachable code after return/raise statements. Unreachable blocks are those that cannot be reached through any execution path.
6. Identify potential infinite loops (loops with incrementing variables)

## Implicit Requirements

1. Handle syntax errors gracefully
2. Process nested control structures correctly
3. Account for early returns and exception handling
4. Distinguish between different types of control flow statements
5. Handle empty input and edge cases

## Edge Cases
1. Empty functions or classes
2. Nested try/except blocks with different exception types

## Constraints
- Maximum 50 lines of input code
- Each line has at most 100 characters
- Code may contain syntax errors
- Only analyze function definitions, ignore module-level code
- Consider 'pass' statements as empty blocks
- Infinite loop detection should be conservative (flag potential cases)
- The tool should work with Python 3.x syntax


In [ ]:
# code

import ast


def analyze_control_flow(source_lines):
    """
    Analyze Python source code and return control flow metrics.

    Args:
        source_lines: List of strings representing Python source code

    Returns:
        tuple: (blocks_count, decision_points, max_depth, unreachable_blocks,
                loop_status)
    """
    # Parse the source code into an Abstract Syntax Tree (AST)
    # Handle potential syntax errors gracefully
    try:
        tree = ast.parse("\n".join(source_lines))
    except SyntaxError:
        # If the code cannot be parsed, return safe defaults
        return (0, 0, 0, "NONE", "FINITE")

    # Initialize global counters and tracking variables
    block_counter = 0  # Counter for assigning unique block IDs
    decision_points = 0  # Counter for branching constructs
    max_depth = 0  # Maximum nesting depth encountered
    unreachable_blocks = set()  # Set of unreachable block IDs
    loop_status = "FINITE"  # Status of loops (FINITE or INFINITE)

    def new_block_id():
        """Generate a new unique block ID and increment counter."""
        nonlocal block_counter
        bid = block_counter
        block_counter += 1
        return bid

    def body_forcibly_ends(body):
        """
        Check if a body of statements forcibly ends execution in all paths.

        Returns True if the body definitely ends (return or raise) for all
        possible execution paths.

        Args:
            body: List of AST statements

        Returns:
            bool: True if all paths end forcibly, False otherwise
        """
        # Empty body doesn't forcibly end
        if not body:
            return False

        # Check for single return/raise statement
        if len(body) == 1 and isinstance(body[0], (ast.Return, ast.Raise)):
            return True

        # Check if first statement is an if-elif-else chain that covers all paths
        if isinstance(body[0], ast.If):
            if_node = body[0]

            # Collect all branches in the if-elif-else chain
            branches = []

            def collect_if_branches(node):
                """Recursively collect all if-elif-else branches."""
                # Add the if/elif body
                branches.append(node.body)

                # Check if there's an elif (represented as nested If in orelse)
                if len(node.orelse) == 1 and isinstance(node.orelse[0], ast.If):
                    # This is an elif - recurse
                    collect_if_branches(node.orelse[0])
                else:
                    # This is the final else clause
                    if node.orelse:
                        branches.append(node.orelse)

            collect_if_branches(if_node)

            # Need at least 2 branches to have complete coverage (if + else)
            if len(branches) < 2:
                return False

            # Check if each branch forcibly ends
            for branch in branches:
                if not body_forcibly_ends(branch):
                    return False

            # If there are statements after the if-elif-else, execution can't
            # reach them, so they would be unreachable
            if len(body) > 1:
                return False

            return True

        return False

    def count_elif_else(if_node):
        """
        Count decision points in an if-elif-else chain.

        Returns the total number of decision points:
        - 1 for the initial if
        - 1 for each elif
        - 1 for the else (if present)

        Args:
            if_node: AST If node

        Returns:
            int: Number of decision points
        """
        # Start with 1 for the initial if statement
        count = 1

        def traverse_orelse(node):
            """Recursively traverse elif/else chain."""
            nonlocal count

            # Check if orelse contains an elif (represented as nested If)
            if len(node.orelse) == 1 and isinstance(node.orelse[0], ast.If):
                # This is an elif - add 1 and recurse
                count += 1
                traverse_orelse(node.orelse[0])
            else:
                # This is a final else clause - add 1 if present
                if node.orelse:
                    count += 1

        traverse_orelse(if_node)
        return count

    def is_while_infinite(while_node):
        """
        Detect potential infinite loops using naive pattern matching.

        Looks for patterns like:
            while True:
            while x > 0:
                x += 1

        Args:
            while_node: AST While node

        Returns:
            bool: True if potentially infinite loop detected
        """
        # Check for while True pattern
        if isinstance(while_node.test, ast.Constant) and while_node.test.value is True:
            return True

        # Check for while True pattern (older Python versions use ast.NameConstant)
        if hasattr(ast, 'NameConstant') and isinstance(while_node.test, ast.NameConstant) and while_node.test.value is True:
            return True

        # Check if the loop condition is a comparison
        if isinstance(while_node.test, ast.Compare):
            comp = while_node.test

            # Look for pattern: variable > constant
            if (isinstance(comp.left, ast.Name) and
                len(comp.ops) == 1 and
                len(comp.comparators) == 1 and
                isinstance(comp.comparators[0], ast.Constant)):

                var_name = comp.left.id  # Variable being compared
                op = comp.ops[0]  # Comparison operator
                cval = comp.comparators[0].value  # Constant value

                # Check for greater than comparison with numeric constant
                if (isinstance(op, (ast.Gt, ast.GtE)) and
                    isinstance(cval, (int, float))):

                    # Search loop body for increments of the same variable
                    for stmt in while_node.body:
                        # Check for augmented assignment (var += value)
                        if isinstance(stmt, ast.AugAssign):
                            if (isinstance(stmt.target, ast.Name) and
                                stmt.target.id == var_name and
                                isinstance(stmt.op, ast.Add)):

                                # Check if increment is positive constant
                                if (isinstance(stmt.value, ast.Constant) and
                                    isinstance(stmt.value.value, (int, float)) and
                                    stmt.value.value > 0):
                                    return True

                        # Check for regular assignment (var = var + value)
                        elif isinstance(stmt, ast.Assign):
                            if (len(stmt.targets) == 1 and
                                isinstance(stmt.targets[0], ast.Name) and
                                stmt.targets[0].id == var_name):

                                # Check if RHS is var + positive_constant
                                if isinstance(stmt.value, ast.BinOp):
                                    binop = stmt.value
                                    if (isinstance(binop.op, ast.Add) and
                                        isinstance(binop.left, ast.Name) and
                                        binop.left.id == var_name and
                                        isinstance(binop.right, ast.Constant) and
                                        isinstance(binop.right.value, (int, float)) and
                                        binop.right.value > 0):
                                        return True
        return False

    def visit_statements(stmts, depth=1, in_unreachable=False):
        """
        Recursively traverse and analyze a list of statements.

        Args:
            stmts: List of AST statements to analyze
            depth: Current nesting depth
            in_unreachable: Whether we're in unreachable code

        Returns:
            bool: True if this block forcibly ends execution
        """
        nonlocal max_depth, decision_points, loop_status

        forced_end_here = False
        i = 0

        # Process each statement in the list
        while i < len(stmts):
            node = stmts[i]

            # Create a new block for this statement
            current_block = new_block_id()

            # Mark as unreachable if we're in unreachable code
            if in_unreachable:
                unreachable_blocks.add(current_block)

            # Update maximum depth tracking
            if depth > max_depth:
                max_depth = depth

            # Handle different types of statements
            if isinstance(node, ast.Return) or isinstance(node, ast.Raise):
                # This block forcibly ends execution
                forced_end_here = True

                # Mark all subsequent statements as unreachable
                i += 1
                while i < len(stmts):
                    unreachable_block = new_block_id()
                    unreachable_blocks.add(unreachable_block)
                    i += 1
                break

            elif isinstance(node, ast.If):
                # Count decision points for if-elif-else chain
                decision_point_count = count_elif_else(node)
                decision_points += decision_point_count

                # Collect forced-end status for each branch
                def collect_if_forced_end(n, d):
                    """Collect forced-end status for all branches."""
                    results = []

                    # Analyze if branch
                    if_end = visit_statements(n.body, d + 1,
                                            in_unreachable=False)
                    results.append(if_end)

                    # Handle elif/else chain
                    if (len(n.orelse) == 1 and
                        isinstance(n.orelse[0], ast.If)):
                        # This is an elif - recurse
                        results.extend(collect_if_forced_end(n.orelse[0], d))
                    else:
                        # This is a final else clause
                        if n.orelse:
                            else_end = visit_statements(n.orelse, d + 1,
                                                      in_unreachable=False)
                            results.append(else_end)

                    return results

                # Get forced-end status for all branches
                forced_branch_results = collect_if_forced_end(node, depth)

                # Check if there's a final else clause
                def has_final_else(n):
                    """Check if if-elif chain has a final else."""
                    if not n.orelse:
                        return False
                    if (len(n.orelse) == 1 and
                        isinstance(n.orelse[0], ast.If)):
                        return has_final_else(n.orelse[0])
                    return True

                has_else = has_final_else(node)

                # If all branches forcibly end and there's an else,
                # code after this if-elif-else is unreachable
                if has_else and all(forced_branch_results):
                    forced_end_here = True

                    # Mark subsequent statements as unreachable
                    i += 1
                    while i < len(stmts):
                        unreachable_block = new_block_id()
                        unreachable_blocks.add(unreachable_block)
                        i += 1
                    break

            elif isinstance(node, ast.For):
                # For loop is a decision point
                decision_points += 1

                # Analyze loop body
                visit_statements(node.body, depth + 1, in_unreachable=False)

                # Analyze else clause (executes if loop completes normally)
                visit_statements(node.orelse, depth + 1, in_unreachable=False)

            elif isinstance(node, ast.While):
                # While loop is a decision point
                decision_points += 1

                # Check for potential infinite loop
                if loop_status == "FINITE" and is_while_infinite(node):
                    loop_status = "INFINITE"

                # Analyze loop body
                visit_statements(node.body, depth + 1, in_unreachable=False)

                # Analyze else clause (executes if loop completes normally)
                visit_statements(node.orelse, depth + 1, in_unreachable=False)

            elif isinstance(node, ast.Try):
                # Try block is a decision point
                decision_points += 1

                # Each except handler is also a decision point
                for handler in node.handlers:
                    decision_points += 1

                # Analyze try body
                visit_statements(node.body, depth + 1, in_unreachable=False)

                # Analyze exception handlers
                for handler in node.handlers:
                    visit_statements(handler.body, depth + 1,
                                   in_unreachable=False)

                # Analyze else clause (executes if no exception)
                visit_statements(node.orelse, depth + 1, in_unreachable=False)

                # Analyze finally clause (always executes)
                visit_statements(node.finalbody, depth + 1,
                               in_unreachable=False)

            else:
                # Regular statement (assignment, expression, etc.)
                # No special handling needed
                pass

            i += 1

        return forced_end_here

    # Analyze all function definitions in the AST
    for node in tree.body:
        if isinstance(node, ast.FunctionDef):
            # Create a block for the function definition itself
            func_block = new_block_id()

            # Recursively analyze the function body
            visit_statements(node.body, depth=1, in_unreachable=False)

    # Format unreachable blocks for output
    if len(unreachable_blocks) == 0:
        unreachable_str = "NONE"
    elif len(unreachable_blocks) == 1:
        # FIXED: Return int for single unreachable block
        unreachable_str = list(unreachable_blocks)[0]
    else:
        # Sort block IDs and join with spaces for multiple blocks
        unreachable_str = " ".join(str(b) for b in sorted(unreachable_blocks))

    # Return the analysis results
    return (
        block_counter,  # Total number of blocks
        decision_points,  # Total number of decision points
        max_depth if max_depth > 0 else 0,  # Maximum nesting depth
        unreachable_str,  # Unreachable block IDs or "NONE"
        loop_status  # "FINITE" or "INFINITE"
    )


def driver_code(source_lines):
    """
    Driver function that formats the analysis results for output.

    Args:
        source_lines: List of strings representing Python source code

    Returns:
        str: Formatted output string matching problem requirements
    """
    # Get analysis results
    (blocks_count, decision_points, max_depth,
     unreachable_blocks, loop_status) = analyze_control_flow(source_lines)

    # Format output according to problem specification
    if unreachable_blocks == "NONE":
        out_str = (f"({blocks_count}, {decision_points}, {max_depth}, "
                  f"NONE, {loop_status})")
    else:
        out_str = (f"({blocks_count}, {decision_points}, {max_depth}, "
                  f"{unreachable_blocks}, {loop_status})")

    return out_str


# Self-test examples (uncomment to run)
if __name__ == "__main__":
    # Example 1: Basic if-else with unreachable code
    src1 = [
        "def calculate(x):",
        "    if x > 0:",
        "        return x * 2",
        "    else:",
        "        return 0",
        "    print(\"unreachable\")"
    ]
    print(driver_code(src1))
    # Expected: (5, 2, 2, 4, FINITE)

    # Example 2: For loop with try-except
    src2 = [
        "def process_data(items):",
        "    for item in items:",
        "        if not item:",
        "            continue",
        "        try:",
        "            result = item.process()",
        "        except:",
        "            pass",
        "    return \"done\""
    ]
    print(driver_code(src2))
    # Expected: (8, 4, 3, NONE, FINITE)

    # Example 3: Infinite while loop
    src3 = [
        "def loop_test():",
        "    x = 1",
        "    while x > 0:",
        "        x += 1",
        "    return x"
    ]
    print(driver_code(src3))
    # Expected: (5, 1, 2, NONE, INFINITE)


In [ ]:
# tests

"""
Control Flow Analyzer Test Suite.

Target Model: GPT-4o (Model Breaking)
Focus: Security vulnerability detection in code analysis
"""

import unittest
from main import analyze_control_flow


class TestControlFlowAnalyzer(unittest.TestCase):
    """Test suite for control flow analyzer with security focus."""

    def test_example_unreachable_code(self):
        """Example 1: Unreachable code after if/else returns."""
        source = [
            "def calculate(x):",
            "    if x > 0:",
            "        return x * 2",
            "    else:",
            "        return 0",
            "    print('unreachable')",
        ]
        self.assertEqual(analyze_control_flow(source), (5, 2, 2, 4, "FINITE"))

    def test_example_complex_nesting(self):
        """Example 2: Complex nested control structures."""
        source = [
            "def process_data(items):",
            "    for item in items:",
            "        if not item:",
            "            continue",
            "        try:",
            "            result = item.process()",
            "        except:",
            "            pass",
            "    return 'done'",
        ]
        self.assertEqual(
            analyze_control_flow(source), (8, 4, 3, "NONE", "FINITE")
        )

    def test_example_infinite_loop(self):
        """Example 3: Infinite loop with incrementing variable."""
        source = [
            "def loop_test():",
            "    x = 1",
            "    while x > 0:",
            "        x += 1",
            "    return x",
        ]
        self.assertEqual(
            analyze_control_flow(source), (5, 1, 2, "NONE", "INFINITE")
        )

    def test_basic_simple_function(self):
        """Basic function with no control structures."""
        source = ["def simple():", "    x = 42", "    return x"]
        self.assertEqual(
            analyze_control_flow(source), (3, 0, 1, "NONE", "FINITE")
        )

    def test_basic_empty_function(self):
        """Empty function with pass statement."""
        source = ["def empty():", "    pass"]
        self.assertEqual(
            analyze_control_flow(source), (2, 0, 1, "NONE", "FINITE")
        )

    def test_exception_bypass_paths(self):
        """Exception bypass paths detection (security vulnerability)."""
        source = [
            "def risky_function():",
            "    try:",
            "        dangerous_operation()",
            "        return 'success'",
            "    except:",
            "        pass",
            "    return 'failed'",
        ]
        result = analyze_control_flow(source)
        self.assertIsInstance(result, tuple)
        self.assertEqual(len(result), 5)

    def test_nested_exception_types(self):
        """Nested try/except blocks with different exception types."""
        source = [
            "def complex_exception():",
            "    try:",
            "        try:",
            "            risky_operation()",
            "        except ValueError:",
            "            handle_value_error()",
            "    except TypeError:",
            "        handle_type_error()",
            "    return 'done'",
        ]
        result = analyze_control_flow(source)
        decision_points, max_depth = result[1], result[2]
        self.assertGreaterEqual(decision_points, 2)
        self.assertGreaterEqual(max_depth, 3)

    def test_decision_if_elif_else(self):
        """If/elif/else decision points counting."""
        source = [
            "def grade(score):",
            "    if score >= 90:",
            "        return 'A'",
            "    elif score >= 80:",
            "        return 'B'",
            "    else:",
            "        return 'F'",
        ]
        result = analyze_control_flow(source)
        self.assertEqual(result[1], 3)

    def test_decision_nested_loops(self):
        """Nested for loops with depth calculation."""
        source = [
            "def nested():",
            "    for i in range(2):",
            "        for j in range(2):",
            "            print(i, j)",
            "    return 'done'",
        ]
        result = analyze_control_flow(source)
        self.assertEqual(result[2], 3)

        """Try/except/finally as decision points."""
        source = [
            "def error_handling():",
            "    try:",
            "        risky()",
            "    except ValueError:",
            "        handle_error()",
            "    finally:",
            "        cleanup()",
            "    return 'done'",
        ]
        result = analyze_control_flow(source)
        self.assertGreaterEqual(result[1], 2)

    def test_unreachable_after_return(self):
        """Code unreachable after return statement."""
        source = [
            "def early_return():",
            "    return 'early'",
            "    print('unreachable')",
        ]
        self.assertEqual(analyze_control_flow(source), (3, 0, 1, 2, "FINITE"))

        """Code unreachable after raise statement."""
        source = [
            "def error_func():",
            "    raise ValueError('error')",
            "    return 'unreachable'",
        ]
        self.assertEqual(analyze_control_flow(source), (3, 0, 1, 2, "FINITE"))

    def test_infinite_while_true(self):
        """While True infinite loop detection."""
        source = [
            "def infinite():",
            "    while True:",
            "        print('forever')",
            "    return 'never'",
        ]
        self.assertEqual(
            analyze_control_flow(source), (4, 1, 2, "NONE", "INFINITE")
        )

    def test_infinite_finite_decrementing(self):
        """Finite loop with decrementing variable (contrast test)."""
        source = [
            "def countdown():",
            "    x = 10",
            "    while x > 0:",
            "        x -= 1",
            "    return x",
        ]
        self.assertEqual(
            analyze_control_flow(source), (5, 1, 2, "NONE", "FINITE")
        )

    def test_edge_empty_classes(self):
        """Empty classes should be ignored."""
        source = [
            "class EmptyClass:",
            "    pass",
            "class AnotherClass:",
            "    def method(self):",
            "        pass",
            "def actual_function():",
            "    return 42",
        ]
        result = analyze_control_flow(source)
        self.assertLessEqual(result[0], 3)

    def test_edge_break_continue_control(self):
        """Break and continue loop control statements."""
        source = [
            "def loop_controls():",
            "    for i in range(10):",
            "        if i == 5:",
            "            break",
            "        if i % 2 == 0:",
            "            continue",
            "        print(i)",
        ]
        result = analyze_control_flow(source)
        self.assertGreaterEqual(result[1], 2)

    def test_edge_multiple_functions(self):
        """Multiple function definitions in same input."""
        source = [
            "def first_function():",
            "    return 1",
            "def second_function():",
            "    if True:",
            "        return 2",
            "    return 0",
        ]
        result = analyze_control_flow(source)
        self.assertGreaterEqual(result[0], 5)

    def test_edge_empty_input(self):
        """Empty input handling."""
        self.assertEqual(analyze_control_flow([]), (0, 0, 0, "NONE", "FINITE"))

    def test_edge_module_level_ignored(self):
        """Module-level code should be ignored."""
        source = [
            "import os",
            "x = 42",
            "print('module level')",
            "def actual_function():",
            "    return x",
        ]
        result = analyze_control_flow(source)
        self.assertLessEqual(result[0], 3)

    def test_edge_syntax_error(self):
        """Graceful handling of syntax errors."""
        source = ["def broken():", "    if x > 0", "        return x"]
        result = analyze_control_flow(source)

        self.assertIsInstance(result, tuple)
        self.assertEqual(len(result), 5)
        (
            blocks_count,
            decision_points,
            max_depth,
            unreachable_blocks,
            loop_status,
        ) = result
        self.assertIsInstance(blocks_count, int)
        self.assertIsInstance(decision_points, int)
        self.assertIsInstance(max_depth, int)
        self.assertIn(type(unreachable_blocks), [int, str])
        if isinstance(unreachable_blocks, str):
            self.assertEqual(unreachable_blocks, "NONE")
        self.assertIn(loop_status, ["INFINITE", "FINITE"])


if __name__ == "__main__":
    unittest.main(verbosity=2)


# Model Breaking Proof

#### Model Breaking Task URL: https://labeling-ms.turing.com/conversations/67448/view

#### Model code:

```python
import ast

class CFGAnalyzer(ast.NodeVisitor):
    def __init__(self):
        self.blocks_count = 0
        self.decision_points = 0
        self.nesting_stack = []
        self.unreachable_blocks = set()
        self.current_block_id = 0
        self.max_depth = 0
        self.infinite_loop = False
    
    def visit_FunctionDef(self, node):
        # Analyze each function independently
        self.nesting_stack.append(0)
        self.generic_visit(node)
        self.nesting_stack.pop()

    def visit_If(self, node):
        self.decision_points += 1
        self._increase_nesting()
        self.generic_visit(node)
        self._decrease_nesting()

    def visit_While(self, node):
        self.decision_points += 1
        # <Issue>: Model's infinite loop detection is too simplistic; only checks 'while True' literals
        # misses cases like while x > 0 where x keeps increasing
        if isinstance(node.test, ast.Constant) and node.test.value != False:
            # Simple infinite loop check for 'while True' or similar
            self.infinite_loop = True
        self._increase_nesting()
        self.generic_visit(node)
        self._decrease_nesting()

    def visit_For(self, node):
        self.decision_points += 1
        self._increase_nesting()
        self.generic_visit(node)
        self._decrease_nesting()

    def visit_Try(self, node):
        self.decision_points += 1
        self._increase_nesting()
        self.generic_visit(node)
        self._decrease_nesting()

    def visit_Return(self, node):
        # <Issue>: Model's unreachable detection logic is fundamentally flawed
        self._mark_unreachable_following_node(node)
    
    def visit_Raise(self, node):
        # <Issue>: Same unreachable detection issue as Return
        self._mark_unreachable_following_node(node)

    def _increase_nesting(self):
        if self.nesting_stack:
            self.nesting_stack[-1] += 1
        current_depth = sum(self.nesting_stack)
        if current_depth > self.max_depth:
            self.max_depth = current_depth
        self.current_block_id += 1
        self.blocks_count += 1
    
    def _decrease_nesting(self):
        if self.nesting_stack and self.nesting_stack[-1] > 0:
            self.nesting_stack[-1] -= 1

    def _mark_unreachable_following_node(self, node):
        # <Issue>: Wrong implementation; current_block_id doesn't represent the actual unreachable block
        self.unreachable_blocks.add(self.current_block_id)

    def generic_visit(self, node):
        # <Issue>: Every AST node is counted as a block, causing massive overcounting
        # Every node accounts for a block
        self.blocks_count += 1
        super().generic_visit(node)

def analyze_control_flow(source_lines):
    source_code = "\n".join(source_lines)
    try:
        tree = ast.parse(source_code)
    except SyntaxError:
        return (0, 0, 0, "NONE", "FINITE")

    analyzer = CFGAnalyzer()
    analyzer.visit(tree)
    
    # Handle unreachable_blocks
    if not analyzer.unreachable_blocks:
        unreachable = "NONE"
    else:
        unreachable = ", ".join(map(str, analyzer.unreachable_blocks))
    
    loop_status = "INFINITE" if analyzer.infinite_loop else "FINITE"

    return (
        analyzer.blocks_count,
        analyzer.decision_points,
        analyzer.max_depth,
        unreachable,
        loop_status
    )

# <Issue>: Model failed to provide a driver function that formats the output correctly
# to test, you may use the driver function (and main) from ideal code

```